In [ ]:
from minio.error import S3Error
import fitz  # PyMuPDF
from datetime import datetime
from io import BytesIO
import pandas as pd
import re
import psycopg2
import import_ipynb
from Utils import conectar_minio,transformate_df, generar_ruta_fecha,extraer_archivo_minio, insert_into_info_lab, cargar_dfpaciente, agregar_fuente, agregar_observacion
import os
from dotenv import load_dotenv

In [ ]:
# Cargar variables del archivo .env
load_dotenv()

In [ ]:
minio_client = conectar_minio(
    endpoint=os.getenv("MINIO_ENDPOINT"),
    access_key=os.getenv("MINIO_ACCESS_KEY"),
    secret_key=os.getenv("MINIO_SECRET_KEY"),
    secure=False  # Usar HTTPS
)

Conexión exitosa a MinIO en localhost:9000
Buckets disponibles: ['yachay', 'yachay-bronze', 'yachay-landing']


In [50]:
ruta_fecha = generar_ruta_fecha()  

In [51]:
 #3. Extraer un archivo Excel/CSV/Parquet
df_datos = extraer_archivo_minio(
    minio_client=minio_client,
    bucket_name="yachay-bronze",
    ruta_archivo=f"laboratorios/{ruta_fecha}/laboratorio Fernando Sanzon/Fernando Sanzon.csv",
    tipo_archivo="csv"
)
print(df_datos)  # Mostrar las primeras filas del DataFrame extraído

                      Archivo Numero de Informe  \
0  R0036-1089242582 CMVA.docx         24FS0036.   
1     R032-59805655 CMVA.docx         24FS0032.   
2         R033-5285368SC.docx          24FS033.   
3        R034-30711337SC.docx          24FS034.   
4       R35 -59834885 SC.docx            24FS35   

                 Nombre del Paciente  Identificacion  Edad  Telefono  \
0      CRIOLLO CAICEDO BLANCA NIDIA.    1.089243e+09    36       NaN   
1                MARTINEZ MARIA LUZ.    5.980566e+07    56       NaN   
2  FRANCISCO PARMENIO ROJAS ALVAREZ.    5.285368e+06    69       NaN   
3             MARIA ANGELICA MINGAN.    3.071134e+07    73       NaN   
4                 AIDA CAROLINA MERA    5.953938e+07    47       NaN   

  Fecha de Toma de Muestra  Fecha de Ingreso   Fecha de Informe  \
0         Enero 10 de 2024               NaN  Enero 16 de 2024.   
1          Enero 5 de 2024               NaN  Enero 12 de 2024.   
2         Enero 9 de 2024.               NaN  Enero 11 de 2

In [ ]:
df_datos = transformate_df(df_datos)  # Transformar el DataFrame
df_datos = agregar_fuente(df_datos, "109")
df_datos = agregar_observacion(df_datos)
print(df_datos)  # Mostrar las primeras filas del DataFrame transformado

                      archivo Numero de Informe  \
0  R0036-1089242582 CMVA.docx          24FS0036   
1     R032-59805655 CMVA.docx          24FS0032   
2         R033-5285368SC.docx           24FS033   
3        R034-30711337SC.docx           24FS034   
4       R35 -59834885 SC.docx            24FS35   

                Nombre del Paciente NumeroIdentificacion  Edad  Telefono  \
0      CRIOLLO CAICEDO BLANCA NIDIA          10892425820    36       NaN   
1                MARTINEZ MARIA LUZ            598056550    56       NaN   
2  FRANCISCO PARMENIO ROJAS ALVAREZ             52853680    69       NaN   
3             MARIA ANGELICA MINGAN            307113370    73       NaN   
4                AIDA CAROLINA MERA            595393850    47       NaN   

  fecha_toma_muestra  fecha_ingreso fecha_informe               entidad  eps  \
0         10/01/2024            NaN    16/01/2024  C.M. VALLE DE ATRIZ.  NaN   
1         05/01/2024            NaN    12/01/2024  C.M. VALLE DE ATRIZ.  NaN

In [ ]:
connection_params = {
    'dbname': os.getenv("DB_NAME"),
    'user': os.getenv("DB_USER"),
    'password': os.getenv("DB_PASSWORD"),
    'host': os.getenv("DB_HOST"),
    'port':os.getenv("DB_PORT")
}

In [54]:
insert_into_info_lab(df_datos, connection_params)

Se insertaron 5 filas en la tabla 'infolab'.


In [44]:
print(df_datos.columns)  # Mostrar las columnas del DataFrame transformado

Index(['archivo', 'Numero de Informe', 'Nombre del Paciente',
       'NumeroIdentificacion', 'Edad', 'Telefono', 'fecha_toma_muestra',
       'fecha_ingreso', 'fecha_informe', 'entidad', 'eps', 'servicio',
       'muestra_remitida', 'descripcion_macroscopica',
       'descripcion_microscopica', 'diagnostico', 'comentario',
       'TipoIdentificacion', 'id_paciente', 'fuente'],
      dtype='object')


In [45]:
df_datos.rename(columns={
    'Nombre del Paciente': 'nombre1',
    'NumeroIdentificacion': 'identificacion',
    'TipoIdentificacion': 'tipo_de_identificacion',
    'Edad': 'edad',
    'Telefono': 'telefono',
}, inplace=True)

print(df_datos.head())

                      archivo Numero de Informe  \
0  R0036-1089242582 CMVA.docx          24FS0036   
1     R032-59805655 CMVA.docx          24FS0032   
2         R033-5285368SC.docx           24FS033   
3        R034-30711337SC.docx           24FS034   
4       R35 -59834885 SC.docx            24FS35   

                            nombre1 identificacion  edad  telefono  \
0      CRIOLLO CAICEDO BLANCA NIDIA    10892425820    36       NaN   
1                MARTINEZ MARIA LUZ      598056550    56       NaN   
2  FRANCISCO PARMENIO ROJAS ALVAREZ       52853680    69       NaN   
3             MARIA ANGELICA MINGAN      307113370    73       NaN   
4                AIDA CAROLINA MERA      595393850    47       NaN   

  fecha_toma_muestra  fecha_ingreso fecha_informe               entidad  eps  \
0         10/01/2024            NaN    16/01/2024  C.M. VALLE DE ATRIZ.  NaN   
1         05/01/2024            NaN    12/01/2024  C.M. VALLE DE ATRIZ.  NaN   
2         09/01/2024            

In [46]:
df_pacientes = df_datos[['nombre1', 'identificacion', 'tipo_de_identificacion', 'edad', 'telefono', 'id_paciente']].copy()

In [47]:
cargar_dfpaciente(df_pacientes,'paciente', connection_params)

Insertados 1 registros nuevos en paciente.


(True, 1)